In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import csv, random, os, sys, json, requests, re
import numpy as np
import pandas as pd
from pandas import read_csv
import matplotlib.pyplot as plt
import time
from google.cloud import bigquery


safe = [
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_ONLY_HIGH",
    },
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_ONLY_HIGH",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_ONLY_HIGH",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_ONLY_HIGH",
    },
]

## 1. Make batchfile & Upload to BigQuery

In [18]:
file_name = "zeroshot_example.json"
data = pd.read_csv('../data.csv')

In [19]:
zeroshot_promt_txt = """The uploaded text is one of the articles that may be real or fake.
Please Answer whether below article is fake or real. Say nothing but the number 0 or 1.
i.e. Answer 1 if you think the article is real, answer 0 if you think it is fake.


###
Output: (rational) + (0||1)


"""

cot_prompt_txt = """The uploaded text is one of the articles that may be real or fake.
Please Answer whether below article is fake or real. Give a 20-character rationale for why you think that way, and output a 0 and 1 at the end of the sentence.
To Solve this, You have to think step by step. The first step in identifying fake news is evaluating the reliability of the information source. Well-known and verified news organizations are generally more reliable, and their reports can be trusted more than unverified sources.
In addition to source reliability, look at the language used in the content. Fake news often uses sensational or exaggerated language designed to elicit an emotional response. It is also important to check for consistency and accuracy in the information presented; fake news typically includes claims that are either unverified or clearly false. Another critical step is cross-verification,
where check if the same claims are reported by multiple trusted sources.
i.e. rationale + answer 1 if you think the article is real, rationale + answer 0 if you think it is fake.\n Must Keep in mind that the end of a sentence should end with either 0 or 1

###
Output: (rational) + (0||1)


"""


fewshot = """Examples:

Example Article (Real)
Title: "Three women contract HIV from dirty “vampire facials” at unlicensed spa"\\
Content: "Trendy, unproven 'vampire facials' performed at an unlicensed spa in New Mexico left at least three women with HIV infections.
This marks the first time that cosmetic procedures have been associated with an HIV outbreak,
according to a detailed report of the outbreak investigation published today. Ars reported on the cluster last year when state health
officials announced they were still identifying cases linked to the spa despite it being shut down in September 2018.
But today’s investigation report offers more insight into the unprecedented outbreak, which linked five people with HIV infections to the spa and spurred investigators to contact and test nearly 200 other spa clients."\\
Example Article (Fake)\\
Title: "New NFL Combine Drill Tests Player’s Ability To Half-Ass Taping Of Local Sandwich Shop Commercial"
Content: "INDIANAPOLIS—In an effort to gain a fuller picture of prospective players and ensure they would be good fits for the teams that drafted them,
 a new NFL Scouting Combine drill tested a player’s ability to half-ass a taping of a local sandwich shop commercial,
  sources confirmed Monday. As part of our new drill, invited players have been asked to stand stiffly in a jersey and ill-fitting jeans while reading awkwardly from cue cards that encourage customers to come on down to Big Dave’s or Galaxy Sandwich.\\

rationale: When determining the authenticity of a news story, "Three women contract HIV from dirty 'vampire facials' at unlicensed spa" is judged to be real news because it is based on detailed investigative reporting and clear facts from a trusted source 1 . On the other hand, "New NFL Combine Drill Tests Player's Ability To Half-Ass Taping Of Local Sandwich Shop Commercial" is considered fake news because it uses satirical language and an unrealistic claim that the NFL Combine tests players' ability to film commercials 0.


Now, Answer whether below article is fake or real with format below.
Output: (rational) + (0||1)
 

"""

In [20]:
tasks = []

prompt = zeroshot_promt_txt = """The uploaded text is one of the articles that may be real or fake.
Please Answer whether below article is fake or real. Say nothing but the number 0 or 1.
i.e. Answer 1 if you think the article is real, answer 0 if you think it is fake.


###
Output: (rational) + (0||1)


"""


for i in range(len(data)):
    task = {
        "request": {
            "contents": [
                {
                "role": "user",
                    "parts": {
                        "text": prompt + data.iloc[i][0]
                    }
                }
            ],
            "system_instruction": {
                "parts": [
                    {
                        "text": "You are a helpful AI assistant."
                    }
                ]
            },
            "safety_settings": safe
        }
  }
    tasks.append(task)
    
print(f"The length of the tasks is {len(tasks)}")

The length of the tasks is 3138


In [21]:
with open(file_name, 'w') as file:
    for obj in tasks:
        file.write(json.dumps(obj) + '\n')

In [22]:
from google.cloud import bigquery
import os

os.environ.setdefault("GCLOUD_PROJECT", "your-project-id")

client = bigquery.Client()
table_id = 'your-project-id.your_dataset.your_table'
job_config = bigquery.LoadJobConfig(

    schema=[
        bigquery.SchemaField("request", "JSON")
    ],
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    write_disposition=bigquery.WriteDisposition.WRITE_APPEND,

)

- load to google Bigquery

In [23]:
with open(file_name, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_id, job_config=job_config)

In [ ]:
while job.state != "DONE":
    job.reload()
    time.sleep(1)
print(job.result())

- request data format for gemini api

In [25]:
request_data = {
    "displayName": "zeroshot_test", # A name you choose for your job.
    "model": "publishers/google/models/gemini-1.5-pro-001",
    "inputConfig": {
        "instancesFormat":"bigquery",
        "bigquerySource": {
            "inputUri" : "bq://planar-ripsaw-430918-a0.batch_zeroshot2.zeroshot2"
        }
    },
    "outputConfig": {
        "predictionsFormat":"bigquery",
        "bigqueryDestination": {
            "outputUri": "bq://planar-ripsaw-430918-a0.batch_zeroshot2.zeroshot2"
        }
    }
}

with open("request.json", "w") as file:
    json.dump(request_data, file, indent=4)

## 2. Get Results from BigQuery

In [15]:

import os
os.environ.setdefault("GCLOUD_PROJECT", "planar-ripsaw-430918-a0")
client = bigquery.Client()

# BigQuery
query = """
SELECT request, response
FROM `planar-ripsaw-430918-a0.batch_fewshot.fewshot`
"""

client = bigquery.Client()
query_job = client.query(query)
results = query_job.result()


data = []
for row in results:
    data.append({
        'request': row.get('request'),
        'response': row.get('response')
    })

# save JSON
with open('fewshot_res.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=4)

print("done")



done


In [8]:
table = client.get_table(table_id)

print("Schema:")
for schema_field in table.schema:
    print(f"{schema_field.name} ({schema_field.field_type})")

Schema:
request (STRING)
response (STRING)
status (STRING)
processed_time (TIMESTAMP)
